# 네이버 뉴스 API를 활용한 데이터 수집 실습

## 들어가며
이 실습에서는 네이버 뉴스 검색 API를 활용하여 실시간으로 뉴스 데이터를 수집하고 분석하는 방법을 배웁니다. 
데이터 수집의 기초부터 데이터 정제, 저장, 그리고 기본적인 분석까지의 전 과정을 파이썬으로 구현해보면서 
실제 데이터 과학 프로젝트의 흐름을 경험해 볼 수 있습니다.

## 학습 목표
1. **API의 개념과 활용**: 외부 서비스 API를 통한 데이터 수집 방법 이해
2. **HTTP 통신 기초**: 요청과 응답 구조, 헤더와 파라미터 활용법 습득
3. **JSON 데이터 처리**: JSON 형식의 데이터를 파싱하고 활용하는 방법 학습
4. **HTML 처리 기술**: Beautiful Soup을 활용한 HTML 태그 제거 및 텍스트 추출
5. **데이터 저장**: 수집한 데이터를 엑셀 파일로 저장하는 다양한 방법 습득
6. **데이터 분석 기초**: 판다스를 활용한 기본 데이터 분석 및 시각화 경험

## 사전 준비사항
- Python 기본 문법에 대한 이해
- Jupyter Notebook 사용법 숙지
- 필요 라이브러리 설치 (`requests`, `beautifulsoup4`, `pandas`, `openpyxl`)

이 실습을 마치면 여러분은 실제 데이터를 수집하고 처리하는 능력을 갖추게 될 것입니다. 
이러한 기술은 데이터 분석, 머신러닝, 그리고 자연어 처리 분야에서 매우 중요한 기초가 됩니다.

## 1. 필요한 라이브러리 임포트

데이터 수집과 처리를 위해 다음과 같은 라이브러리들을 사용합니다:

- **requests**: 웹 서버에 HTTP 요청을 보내고 응답을 받는 기능을 제공합니다.
- **json**: JSON 형식의 데이터를 파이썬 객체로 변환하거나, 파이썬 객체를 JSON으로 변환하는 기능을 제공합니다.
- **BeautifulSoup**: HTML 또는 XML 문서를 파싱하고 데이터를 추출하는 기능을 제공합니다.
- **pandas**: 데이터 분석과 조작을 위한 고성능 도구를 제공합니다.
- **re**: 정규표현식을 사용하여 텍스트를 처리하는 기능을 제공합니다.
- **openpyxl**: 엑셀 파일(.xlsx)을 읽고 쓰는 기능을 제공합니다.

만약 이러한 라이브러리가 설치되어 있지 않다면, 다음과 같이 설치할 수 있습니다:
```
pip install requests beautifulsoup4 pandas openpyxl
```

In [1]:
# 필요한 라이브러리 임포트
import requests  # HTTP 요청을 보내기 위한 라이브러리
import json  # JSON 데이터 처리를 위한 라이브러리
from bs4 import BeautifulSoup  # HTML 파싱을 위한 라이브러리
import pandas as pd  # 데이터 처리를 위한 라이브러리
import re  # 정규표현식을 사용하기 위한 라이브러리
import openpyxl  # 엑셀 파일 처리를 위한 라이브러리

## 2. API 인증 정보 설정

네이버에서는 개발자들에게 다양한 API 서비스를 제공하고 있습니다. 이러한 API를 사용하기 위해서는 인증 정보가 필요합니다.

### 네이버 API 인증 정보 발급 방법
1. 네이버 개발자 센터(https://developers.naver.com)에 접속합니다.
2. 회원가입 및 로그인 후 "애플리케이션 등록" 메뉴를 선택합니다.
3. "애플리케이션 이름"과 사용할 API를 선택하고 필요한 정보를 입력합니다.
4. 등록 완료 후 "Client ID"와 "Client Secret"을 발급받습니다.

### 중요한 보안 사항
* API 키는 개인 정보와 같이 중요한 보안 정보입니다.
* 실제 프로젝트에서는 코드에 직접 입력하지 말고, 환경 변수나 별도의 설정 파일에서 불러오는 것이 좋습니다.
* Git과 같은 버전 관리 시스템에 API 키를 포함하지 않도록 주의해야 합니다.

**참고**: 이 실습에서는 제공된 API 키를 사용하지만, 실제 프로젝트에서는 본인의 API 키를 발급받아 사용하는 것이 좋습니다.

In [2]:
# 네이버 개발자 센터에서 발급받은 API 인증 정보
client_id = "yR8aViAqV5NyEjZpfo74"  # 네이버 API 클라이언트 ID
client_secret = "SQ_PTt2epk"  # 네이버 API 클라이언트 시크릿

## 3. 검색어 입력 및 API 요청 설정

API를 통해 검색 결과를 가져오기 위해 사용자로부터 검색어를 입력받고, API 요청에 필요한 URL과 헤더를 설정합니다.

### 주요 개념
- **URL 파라미터**: API 요청시 검색어(`query`), 결과 개수(`display`) 등을 URL에 포함시켜 전달합니다.
- **요청 헤더**: API 인증을 위해 클라이언트 ID와 시크릿 키를 헤더에 포함시킵니다.
- **f-string**: 파이썬 3.6 이상에서 제공하는 문자열 포맷팅 방식으로, 변수를 문자열 안에 쉽게 삽입할 수 있습니다.

### 네이버 API 매개변수
- `query`: 검색어 (필수)
- `display`: 검색 결과 출력 건수 (기본값: 10, 최대: 100)
- `start`: 검색 시작 위치 (기본값: 1, 최대: 1000)
- `sort`: 정렬 옵션 (`sim`: 유사도순, `date`: 날짜순)

다음 코드를 통해 사용자로부터 검색어를 입력받고 API 요청을 준비합니다.

In [3]:
# 사용자로부터 검색어 입력 받기
query = input("검색할 키워드를 입력하세요: ")  # 검색할 키워드 입력
display = 100  # 한 번에 가져올 검색 결과 개수 (최대 100개까지 설정 가능)

# 네이버 뉴스 검색 API URL 생성
url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}"

# API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
}

## 4. API 요청 및 응답 처리

설정한 URL과 헤더를 사용하여 API 요청을 보내고, 응답을 처리합니다.

### 코드 설명
- `requests.get()`: HTTP GET 요청을 보내는 함수입니다. URL과 헤더 정보를 인자로 전달합니다.
- `response.status_code`: HTTP 응답 상태 코드로, 요청이 성공했는지 여부를 확인할 수 있습니다.
  - 200: 요청 성공
  - 400: 잘못된 요청
  - 401: 인증 실패
  - 404: 리소스를 찾을 수 없음
  - 500: 서버 내부 오류
- `response.json()`: 응답 데이터를 JSON 형식으로 파싱합니다.

### 예외 처리
API 요청 시에는 항상 응답 상태 코드를 확인하여 요청이 성공적으로 처리되었는지 확인하는 것이 중요합니다. 이를 통해 오류 발생 시 적절한 처리를 할 수 있습니다.

In [4]:
# API 요청 보내기
response = requests.get(url, headers=headers)

# 응답 상태 코드 확인
print(f"응답 상태 코드: {response.status_code}")

# 응답이 성공적인 경우 (상태 코드 200)
if response.status_code == 200:
    # JSON 응답 데이터 파싱
    news_data = response.json()
    
    # 검색 결과 항목 수 출력
    if 'items' in news_data:
        print(f"총 {len(news_data['items'])}개의 뉴스 기사를 찾았습니다.")
    else:
        print("검색 결과가 없습니다.")
else:
    print(f"API 요청 실패: {response.status_code}")

응답 상태 코드: 200
총 100개의 뉴스 기사를 찾았습니다.


## 5. 검색 결과 데이터 확인

API 응답으로 받은 JSON 데이터의 구조를 확인합니다. JSON(JavaScript Object Notation)은 데이터를 교환하는 데 널리 사용되는 형식입니다.

### JSON 데이터 구조 이해하기
네이버 뉴스 API는 다음과 같은 구조의 JSON 데이터를 반환합니다:
- `lastBuildDate`: 검색 결과 생성 시간
- `total`: 검색 결과 총 개수
- `start`: 검색 결과 시작 위치
- `display`: 한 번에 표시되는 검색 결과 개수
- `items`: 뉴스 기사 항목 배열
  - `title`: 뉴스 제목
  - `originallink`: 뉴스 원본 링크
  - `link`: 네이버 뉴스 링크
  - `description`: 뉴스 내용 요약
  - `pubDate`: 발행 날짜

아래 코드에서는 `json.dumps()` 함수를 사용하여 첫 번째 뉴스 항목을 보기 좋게 출력합니다. `indent` 파라미터는 들여쓰기를, `ensure_ascii=False`는 한글이 깨지지 않도록 합니다.

In [5]:
# API 응답 데이터 구조 확인 (첫 번째 항목만)
if response.status_code == 200 and 'items' in news_data and len(news_data['items']) > 0:
    print("첫 번째 뉴스 항목 데이터:")
    print(json.dumps(news_data['items'][0], indent=2, ensure_ascii=False))
else:
    print("데이터를 표시할 수 없습니다.")

첫 번째 뉴스 항목 데이터:
{
  "title": "[채희창칼럼] 날개 없이 추락하는 <b>교권</b>",
  "originallink": "http://www.segye.com/newsView/20250519514994?OutUrl=naver",
  "link": "https://n.news.naver.com/mnews/article/022/0004036830?sid=110",
  "description": "늘어 <b>교권</b> 보호 없인 국가 발전 어려워 지난달 28일 부산의 한 초등학교에서 5학년 A군이 여교사를 폭행하는... 정신적 충격을 받은 B교사가 <b>교권</b>보호위원회를 열려고 하자 A군의 부모는 되레 B교사를 아동학대 혐의로... ",
  "pubDate": "Mon, 19 May 2025 23:30:00 +0900"
}


## 6. 검색 결과 처리 및 엑셀 파일 저장

검색 결과에서 제목과 본문을 추출하고, HTML 태그를 제거한 후 엑셀 파일로 저장합니다.

### 데이터 정제 과정
1. **HTML 강조 태그 제거**: 네이버 검색 API는 검색어와 일치하는 부분을 `<b>` 태그로 강조합니다. 이 태그를 제거하여 텍스트만 추출합니다.
2. **HTML 태그 제거**: BeautifulSoup 라이브러리를 사용하여 남아있는 HTML 태그를 제거하고 순수 텍스트를 추출합니다.

### 엑셀 파일 저장 과정
1. **워크북 생성**: openpyxl을 사용하여 새 엑셀 파일을 생성합니다.
2. **데이터 추가**: 추출한 제목과 본문을 행으로 추가합니다.
3. **파일 저장**: 검색어를 파일명에 포함하여 저장합니다.

이 과정을 통해 웹에서 가져온 데이터를 구조화된 형태로 저장하고 추후 분석에 활용할 수 있습니다.

In [6]:
if response.status_code == 200:
    # Openpyxl을 사용하여 새 워크북 생성
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "네이버 뉴스 검색 결과"
    
    # 열 제목 추가
    ws.append(["제목", "본문"])
    
    if 'items' in news_data:
        # 검색 결과가 있는 경우 처리 시작
        for item in news_data['items']:
            # HTML 강조 태그(<b></b>)를 제거하여 제목과 본문 텍스트 추출
            title = item['title'].replace("<b>", "").replace("</b>", "")
            content = item['description'].replace("<b>", "").replace("</b>", "")
            
            # BeautifulSoup을 사용하여 남아있는 HTML 태그를 제거하고 순수 텍스트 추출
            clean_title = BeautifulSoup(title, "html.parser").text
            clean_content = BeautifulSoup(content, "html.parser").text
            
            # 추출한 제목과 본문을 콘솔에 출력
            print(f"제목: {clean_title}")
            print(f"본문: {clean_content}")
            print("-" * 50)
            
            # 추출한 제목과 본문을 엑셀 워크시트에 추가
            ws.append([clean_title, clean_content])
        
        # 엑셀 파일 저장
        file_name = f"{query}_news.xlsx"
        wb.save(file_name)
        print(f"'{file_name}' 파일에 저장되었습니다.")
    else:
        # 검색 결과가 없는 경우 메시지 출력
        print("검색 결과가 없습니다.")
else:
    # API 요청이 실패한 경우 오류 코드와 함께 메시지 출력
    print("API 요청 실패:", response.status_code)

제목: [채희창칼럼] 날개 없이 추락하는 교권
본문: 늘어 교권 보호 없인 국가 발전 어려워 지난달 28일 부산의 한 초등학교에서 5학년 A군이 여교사를 폭행하는... 정신적 충격을 받은 B교사가 교권보호위원회를 열려고 하자 A군의 부모는 되레 B교사를 아동학대 혐의로... 
--------------------------------------------------
제목: 이재명 후보 '미래교육자치위원회' 공식 출범… 200명 넘는 전문가 집결
본문: 총장), 교권보호특위(김성근 전 교육부 학교혁신지원실장), 학부모소통특위(김유임 전 대통령실... 학생 건강·안전, 교권, 학부모, 지역거버넌스, 교육회복, 특수교육 이외에도 학생건강안전특위는 대학 내 학생상담센터... 
--------------------------------------------------
제목: [교육논단] 교육 공약을 살펴보다
본문: 각종 제도와 장치를 통해서 교권과 학습권을 강화하여 교육 회복을 여는 것. 기초 학력 교과 교육을... 교원지위법에 대한 개정 및 교육활동 중에서 발생하는 안전사고에 대한 교원의 책임을 면책 등 교권 향상에 관한... 
--------------------------------------------------
제목: [앵커 브리핑] 갈수록 추락하는 교권···공교육 회복은 어떻게?
본문: 2023년 서이초 사건 당시 전국의 교사들이 거리로 나와 ‘공교육 멈춤의 날’ 집회가 열렸고 '교권 보호 5법'이 국회를 통과했습니다. 교권 보호 5법의 핵심은 학생 분리 조치 시행과 학교 민원 대응팀 구성입니다.... 
--------------------------------------------------
제목: 비참한 교권, 누가 이렇게 만들었나
본문: 교권은 오랫동안 학생과 학부모, 그리고 그 잘난 정치인들에게 셀 수도 없을 만큼 차였다. 더 이상 떨어질... 이유로는 교권 침해 및 과도한 민원이 48.3%로 가장 많았다. '1년간 교권 침해를 당한 경험이 있는지'에

/tmp/ipykernel_48093/386099584.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_content = BeautifulSoup(content, "html.parser").text


## 7. 판다스를 활용한 데이터 분석 (선택 사항)

저장된 데이터를 판다스 데이터프레임으로 변환하여 추가 분석을 수행할 수 있습니다. 판다스(pandas)는 데이터 분석과 조작을 위한 강력한 파이썬 라이브러리로, 다양한 형태의 데이터를 효과적으로 다룰 수 있게 해줍니다.

### 판다스 데이터프레임의 장점
- 행과 열 구조로 데이터를 직관적으로 표현
- 다양한 데이터 소스에서 데이터 로드 가능 (CSV, Excel, SQL 등)
- 강력한 필터링, 그룹화, 집계 기능 제공
- 시각화 도구와의 손쉬운 연계

### 추가 분석 가능성
1. **텍스트 분석**: 뉴스 본문에 자주 등장하는 키워드 분석
2. **감정 분석**: 뉴스 기사의 긍정/부정 분위기 파악
3. **기사 분류**: 주제에 따른 기사 자동 분류 시스템 구축
4. **트렌드 분석**: 시간에 따른 키워드 변화 추적

다음 코드에서는 수집한 뉴스 데이터를 판다스 데이터프레임으로 변환하고, 필요에 따라 CSV 파일로 저장하는 방법을 보여줍니다.

In [7]:
# 판다스 데이터프레임으로 변환
if response.status_code == 200 and 'items' in news_data:
    # 데이터 준비
    titles = []
    contents = []
    
    for item in news_data['items']:
        title = BeautifulSoup(item['title'].replace("<b>", "").replace("</b>", ""), "html.parser").text
        content = BeautifulSoup(item['description'].replace("<b>", "").replace("</b>", ""), "html.parser").text
        titles.append(title)
        contents.append(content)
    
    # 데이터프레임 생성
    df = pd.DataFrame({
        '제목': titles,
        '본문': contents
    })
    
    # 데이터프레임 출력
    print(df)
    
    # 데이터프레임을 CSV 파일로 저장 (선택 사항)
    # df.to_csv(f"{query}_news.csv", index=False, encoding='utf-8-sig')
    # print(f"'{query}_news.csv' CSV 파일로 저장되었습니다.")

                                           제목  \
0                       [채희창칼럼] 날개 없이 추락하는 교권   
1    이재명 후보 '미래교육자치위원회' 공식 출범… 200명 넘는 전문가 집결   
2                          [교육논단] 교육 공약을 살펴보다   
3         [앵커 브리핑] 갈수록 추락하는 교권···공교육 회복은 어떻게?   
4                         비참한 교권, 누가 이렇게 만들었나   
..                                        ...   
95                    ‘생명의 소중함, 스승의 사랑으로’ 캠페인   
96  교사에 도발하고 녹음해 아동학대 누명…'아이혁신당' 만들어 담임 괴롭...   
97                       경남교총 ‘같이·가치데이’ 행사 열어   
98    이준석 “교사 소송 국가책임제” 교육계 표심 공략…권영국 “기후정...   
99            아산성심학교, 교권 보호 캠페인 및 스승의 날 행사 개최   

                                                   본문  
0   늘어 교권 보호 없인 국가 발전 어려워 지난달 28일 부산의 한 초등학교에서 5학년...  
1   총장), 교권보호특위(김성근 전 교육부 학교혁신지원실장), 학부모소통특위(김유임 전...  
2   각종 제도와 장치를 통해서 교권과 학습권을 강화하여 교육 회복을 여는 것. 기초 학...  
3   2023년 서이초 사건 당시 전국의 교사들이 거리로 나와 ‘공교육 멈춤의 날’ 집회...  
4   교권은 오랫동안 학생과 학부모, 그리고 그 잘난 정치인들에게 셀 수도 없을 만큼 차...  
..                                                ...  
95  남윤제 회장은 “단 한 명의

/tmp/ipykernel_48093/4293506620.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  content = BeautifulSoup(item['description'].replace("<b>", "").replace("</b>", ""), "html.parser").text
